### 1.ライブラリのインポート

In [41]:
#一度pip install したらコメントアウトしてください！！
# !pip install japanize-matplotlib 
# !pip install tabulate
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import sys
import os


### 2.データの読み込みと前処理

In [ ]:
# データの読み込みと前処理 (エンコーディング修正済み)
# ファイル格納場所に応じてパスを変える
# パターン1: work_datasetフォルダ内にある時
path_pattern_1 = 'work_dataset/stats_bat.csv'

# パターン2: 実行ファイルと同じ階層の時
path_pattern_2 = 'stats_bat.csv'

# どちらのパスを使うか決定するための変数
use_file_path = None

# ファイルがあるか確認 
if os.path.exists(path_pattern_1):
    use_file_path = path_pattern_1
    print(f"パターン1のパスを採用: {use_file_path}")

elif os.path.exists(path_pattern_2):
    use_file_path = path_pattern_2
    print(f"パターン2のパスを採用: {use_file_path}")

else:
    # どちらも見つからなかったとき
    print("エラー: 'stats_bat.csv' が見つかりません。")
    print(f"以下の場所を確認してください:")
    print(f" - {path_pattern_1}")
    print(f" - {path_pattern_2}")
    # プログラムをここで終了するか、処理をスキップ
    use_file_path = None


# 読み込み処理 (のちの処理のために変数名を変更)
file_name = use_file_path
try:
    # 最初にShift-JISで読み込みを試みる
    df = pd.read_csv(file_name, encoding='Shift-JIS')
    print(":white_check_mark: Shift-JISでファイルを読み込みました。")
except UnicodeDecodeError:
    try:
        # Shift-JISで失敗した場合、cp932 (WindowsのShift-JIS)で試す
        df = pd.read_csv(file_name, encoding='cp932')
        print(":white_check_mark: cp932でファイルを読み込みました。")
    except Exception as e:
        print(f":x: ファイルの読み込みに失敗しました。エンコーディングエラー: {e}")
        sys.exit() # 読み込みエラーでプログラムを終了

# 必要なカラムを選択
columns_to_keep = ['player_id', 'player_name', 'pa', 'ab', 'h', 'tb', 'bb', 'hbp', 'sf', 'so', 'hr']
df_bat = df[columns_to_keep].copy()

# 規定打席数（今回は100打席以上）未満の選手を除外
MIN_PA = 100
df_bat = df_bat[df_bat['pa'] >= MIN_PA].reset_index(drop=True)

print(f":white_check_mark: データ読み込み完了。規定打席数 ({MIN_PA} PA) 以上の選手数: {len(df_bat)}名")
print("-" * 30)

#dfの確認
df_bat.info()#データフレームの情報(カラム名)の確認

パターン1のパスを採用: work_dataset/stats_bat.csv
:white_check_mark: cp932でファイルを読み込みました。
:white_check_mark: データ読み込み完了。規定打席数 (100 PA) 以上の選手数: 892名
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   player_id    892 non-null    int64 
 1   player_name  892 non-null    object
 2   pa           892 non-null    int64 
 3   ab           892 non-null    int64 
 4   h            892 non-null    int64 
 5   tb           892 non-null    int64 
 6   bb           892 non-null    int64 
 7   hbp          892 non-null    int64 
 8   sf           892 non-null    int64 
 9   so           892 non-null    int64 
 10  hr           892 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 76.8+ KB


### 3.ops計算

In [ ]:
# 出塁率 (OBP) = (H + BB + HBP) / (AB + BB + HBP + SF)
df_bat['OBP_denominator'] = df_bat['ab'] + df_bat['bb'] + df_bat['hbp'] + df_bat['sf']
df_bat['OBP_numerator'] = df_bat['h'] + df_bat['bb'] + df_bat['hbp']
df_bat['OBP'] = np.where(df_bat['OBP_denominator'] > 0, df_bat['OBP_numerator'] / df_bat['OBP_denominator'], np.nan)

# 長打率 (SLG) = TB / AB
df_bat['SLG'] = np.where(df_bat['ab'] > 0, df_bat['tb'] / df_bat['ab'], np.nan)

# OPS = OBP + SLG
df_bat['OPS'] = df_bat['OBP'] + df_bat['SLG']


#df確認
# df_bat.head() #先頭の５行を表示して確認 
df_bat.info()#新しい情報が追加されているか確認

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   player_id        892 non-null    int64  
 1   player_name      892 non-null    object 
 2   pa               892 non-null    int64  
 3   ab               892 non-null    int64  
 4   h                892 non-null    int64  
 5   tb               892 non-null    int64  
 6   bb               892 non-null    int64  
 7   hbp              892 non-null    int64  
 8   sf               892 non-null    int64  
 9   so               892 non-null    int64  
 10  hr               892 non-null    int64  
 11  OBP_denominator  892 non-null    int64  
 12  OBP_numerator    892 non-null    int64  
 13  OBP              892 non-null    float64
 14  SLG              892 non-null    float64
 15  OPS              892 non-null    float64
dtypes: float64(3), int64(12), object(1)
memory usage: 111.6+ KB


### 4.分析指標の計算

In [ ]:
# 三振率 K% = SO / PA
df_bat['K_Rate'] = df_bat['so'] / df_bat['pa']

# ホームラン率 HR/PA = HR / PA
df_bat['HR_Rate'] = df_bat['hr'] / df_bat['pa']

df_bat.dropna(subset=['OPS', 'K_Rate', 'HR_Rate'], inplace=True)

print(":white_check_mark: OPSおよび分析指標の計算完了。")
print("-" * 30)

#df確認
df_bat.head()

### 5.相関分析と可視化
- ※ここでは分析結果の概要出力とグラフのファイル保存のみ行います

#### 5-A. OPSと三振率の関係

In [45]:
# 日本語表示のための設定 (環境によって適宜調整してください)
plt.rcParams['font.family'] = 'Hiragino Maru Gothic Pro'  
plt.rcParams['axes.unicode_minus'] = False # マイナス記号の表示

corr_k_ops = df_bat['K_Rate'].corr(df_bat['OPS'])
print(f"三振率 (K_Rate) と OPS の相関係数: {corr_k_ops:.4f}")
plt.figure(figsize=(8, 6))
sns.scatterplot(x='K_Rate', y='OPS', data=df_bat, alpha=0.6)
plt.title(f'OPSと三振率の関係 (相関係数: {corr_k_ops:.4f})')
plt.xlabel('三振率 (SO / PA)')
plt.ylabel('OPS')
plt.savefig('ops_vs_k_rate.png')
plt.close()

三振率 (K_Rate) と OPS の相関係数: -0.1185


#### 5-B.OPSとホームラン率の関係

In [46]:
corr_hr_ops = df_bat['HR_Rate'].corr(df_bat['OPS'])
print(f"ホームラン率 (HR_Rate) と OPS の相関係数: {corr_hr_ops:.4f}")
plt.figure(figsize=(8, 6))
sns.scatterplot(x='HR_Rate', y='OPS', data=df_bat, alpha=0.6)
plt.title(f'OPSとホームラン率の関係 (相関係数: {corr_hr_ops:.4f})')
plt.xlabel('ホームラン率 (HR / PA)')
plt.ylabel('OPS')
plt.savefig('ops_vs_hr_rate.png')
plt.close()

ホームラン率 (HR_Rate) と OPS の相関係数: 0.6911


#### 5-C.高三振・高OPSバッターの確認

In [ ]:
k_rate_q3 = df_bat['K_Rate'].quantile(0.75)
high_k_high_ops_players = df_bat[df_bat['K_Rate'] >= k_rate_q3].sort_values(by='OPS', ascending=False)
print(f"### 三振率が上位25% (K_Rate >= {k_rate_q3:.4f}) のバッター上位10名 (OPS順) ###")
print(high_k_high_ops_players[['player_name', 'pa', 'so', 'hr', 'K_Rate', 'HR_Rate', 'OBP', 'SLG', 'OPS']].head(10).to_markdown(index=False, floatfmt=".3f"))
print("-" * 30)


### 6.データベースへの挿入

In [ ]:
db_name = 'baseball_analysis.db'
table_name = 'batter_performance_ops'
conn = None

try:
    conn = sqlite3.connect(db_name)
    print(f":white_check_mark: データベース '{db_name}' に接続しました。")

    # 挿入するデータフレームを、必要なカラムだけに絞る
    df_result = df_bat[['player_id', 'player_name', 'pa', 'so', 'hr', 'K_Rate', 'HR_Rate', 'OBP', 'SLG', 'OPS']].copy()
    
    # DBへの書き込み (テーブルが既に存在する場合は置き換える)
    df_result.to_sql(
        name=table_name, 
        con=conn, 
        if_exists='replace', 
        index=False 
    )
    
    print(f":white_check_mark: 分析結果をテーブル '{table_name}' に挿入しました。")
    
    # 確認のために、DBから最初の5行を読み出して表示
    df_check = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 5", conn)
    print("\n--- 挿入されたデータの確認 (最初の5行) ---")
    print(df_check.to_markdown(index=False, floatfmt=".3f"))
    
except Exception as e:
    print(f"データベース処理中にエラーが発生しました: {e}")

finally:
    if conn:
        conn.close()
        print("\n:white_check_mark: データベース接続を閉じました。")